<a href="https://colab.research.google.com/github/MSISCapstone/CSPatchTST/blob/mourya_wpatch_short/waptch_sp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone -b mourya_wpatch_short https://github.com/MSISCapstone/CSPatchTST.git

Cloning into 'CSPatchTST'...
remote: Enumerating objects: 854, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 854 (delta 89), reused 59 (delta 59), pack-reused 741 (from 2)
Receiving objects: 100% (854/854), 42.91 MiB | 13.15 MiB/s, done.
Resolving deltas: 100% (451/451), done.
Filtering content: 100% (2/2), 221.27 MiB | 12.28 MiB/s, done.


In [2]:
%cd /content/CSPatchTST/PatchTST_physics_integrated


/content/CSPatchTST/PatchTST_physics_integrated


In [3]:
!ls

config.py	       evaluation.py  models	 trainer.py
data		       __init__.py    models.py  training_utils.py
data_preprocessing.py  layers	      README.md  utils


In [4]:
!ls models
!ls layers
!ls utils


__init__.py  stage1_input.py  stage2_long.py  stage2_short.py
__init__.py  revin.py
__init__.py  time_features.py


In [5]:
import os, sys
print("CWD:", os.getcwd())
print("Top sys.path:", sys.path[:3])


CWD: /content/CSPatchTST/PatchTST_physics_integrated
Top sys.path: ['/content', '/env/python', '/usr/lib/python312.zip']


In [6]:
%cd /content/CSPatchTST
!ls


/content/CSPatchTST
datasets   OutputCharts			pic
docs	   PatchTST_physics_integrated	README.md
LICENSE    PatchTST_self_supervised	waptch_sp.ipynb
notebooks  PatchTST_supervised		Wpatch_SP.ipynb


In [7]:
import sys
sys.path.insert(0, "/content/CSPatchTST")


In [8]:
import PatchTST_physics_integrated
print("Loaded from:", PatchTST_physics_integrated.__file__)


Loaded from: /content/CSPatchTST/PatchTST_physics_integrated/__init__.py


In [9]:
import pandas as pd

CSV_PATH = "/content/CSPatchTST/datasets/weather/weather.csv"
df = pd.read_csv(CSV_PATH)

print("Total columns:", len(df.columns))
for c in df.columns:
    print(repr(c))


Total columns: 22
'date'
'p (mbar)'
'T (degC)'
'Tpot (K)'
'Tdew (degC)'
'rh (%)'
'VPmax (mbar)'
'VPact (mbar)'
'VPdef (mbar)'
'sh (g/kg)'
'H2OC (mmol/mol)'
'rho (g/m**3)'
'wv (m/s)'
'max. wv (m/s)'
'wd (deg)'
'rain (mm)'
'raining (s)'
'SWDR (W/m�)'
'PAR (�mol/m�/s)'
'max. PAR (�mol/m�/s)'
'Tlog (degC)'
'OT'


In [10]:
import re

def find_col(pattern):
    pat = re.compile(pattern, re.IGNORECASE)
    matches = [c for c in df.columns if pat.search(c)]
    return matches[0] if matches else None

col_date = find_col(r"^date$") or find_col(r"date")

# Required core cols (these usually match exactly, but we still match loosely)
col_p     = find_col(r"^p\b") or "p (mbar)"
col_T     = find_col(r"^T\s*\(") or "T (degC)"
col_Tpot  = find_col(r"Tpot")
col_Tdew  = find_col(r"Tdew")
col_rh    = find_col(r"\brh\b")
col_vpmax = find_col(r"VPmax")
col_vpact = find_col(r"VPact")
col_vpdef = find_col(r"VPdef")
col_sh    = find_col(r"^sh\b")
col_h2oc  = find_col(r"H2OC")
col_rho   = find_col(r"\brho\b")
col_wv    = find_col(r"^wv\b")
col_maxwv = find_col(r"max\.\s*wv")
col_wd    = find_col(r"^wd\b")
col_rain  = find_col(r"^rain\b")
col_raining = find_col(r"raining")

# Radiation/light cols (these are the ones failing)
col_swdr   = find_col(r"SWDR|solar|shortwave|rad")
col_par    = find_col(r"\bPAR\b(?!max)")
col_maxpar = find_col(r"max\.\s*PAR|max\s*PAR")

# Optional log temp
col_tlog = find_col(r"Tlog")

feature_cols = [
    col_p, col_T, col_Tpot, col_Tdew, col_rh,
    col_vpmax, col_vpact, col_vpdef, col_sh, col_h2oc,
    col_rho, col_wv, col_maxwv, col_wd, col_rain,
    col_raining
]

# Add radiation cols only if present
for c in [col_swdr, col_par, col_maxpar]:
    if c is not None:
        feature_cols.append(c)

# Add Tlog if present
if col_tlog is not None:
    feature_cols.append(col_tlog)

print("Selected feature cols:", feature_cols)
missing = [c for c in feature_cols if c not in df.columns]
print("Missing after selection:", missing)
print("Total selected:", len(feature_cols))


Selected feature cols: ['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)', 'wd (deg)', 'rain (mm)', 'raining (s)', 'SWDR (W/m�)', 'PAR (�mol/m�/s)', 'max. PAR (�mol/m�/s)', 'Tlog (degC)']
Missing after selection: []
Total selected: 20


In [11]:
import pandas as pd
import numpy as np
import torch

CSV_PATH = "/content/CSPatchTST/datasets/weather/weather.csv"
df = pd.read_csv(CSV_PATH)

df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)

feature_cols = [
    'p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
    'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)',
    'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)', 'wd (deg)', 'rain (mm)',
    'raining (s)', 'SWDR (W/m�)', 'PAR (�mol/m�/s)', 'max. PAR (�mol/m�/s)', 'Tlog (degC)'
]

X_all = df[feature_cols].astype(np.float32).values          # [N,20]
hour_all = df["date"].dt.hour.astype(np.int64).values       # [N]

L = 168
N = len(df) - L + 1

X_weather = torch.from_numpy(np.stack([X_all[i:i+L] for i in range(N)], axis=0))   # [B,L,20]
hour_idx  = torch.from_numpy(np.stack([hour_all[i:i+L] for i in range(N)], axis=0))# [B,L]

print(X_weather.shape, hour_idx.shape)


torch.Size([52529, 168, 20]) torch.Size([52529, 168])


In [12]:
from PatchTST_physics_integrated.models.stage1_input import Stage1Input

stage1 = Stage1Input()
X_long, X_short, X_aux = stage1(X_weather, hour_idx)

print("X_long :", X_long.shape)   # [B,L,15]
print("X_short:", X_short.shape)  # [B,L,5]
print("X_aux  :", X_aux.shape)    # [B,L,2]


X_long : torch.Size([52529, 168, 15])
X_short: torch.Size([52529, 168, 5])
X_aux  : torch.Size([52529, 168, 2])


In [13]:
!sed -n '1,20p' /content/CSPatchTST/PatchTST_physics_integrated/models/stage2_long.py


import torch
import torch.nn as nn
import torch.nn.functional as F


class ReplicationPad1dPatchify(nn.Module):
    """
    Long-channel path: ReplicationPad1d -> unfold into patches.
    Input : X_long [B, L, C_long]
    Output: patches [B, C_long, Np, patch_len]
    """
    def __init__(self, patch_len: int, stride: int):
        super().__init__()
        self.patch_len = int(patch_len)
        self.stride = int(stride)

    def forward(self, x_long: torch.Tensor) -> torch.Tensor:
        # x_long: [B, L, C]
        B, L, C = x_long.shape



In [14]:
%cd /content/CSPatchTST
import sys
sys.path.insert(0, "/content/CSPatchTST")

from PatchTST_physics_integrated.models.stage2_long import ReplicationPad1dPatchify

patchify_long = ReplicationPad1dPatchify(patch_len=48, stride=24)
patches_long = patchify_long(X_long)

print(patches_long.shape)


/content/CSPatchTST
torch.Size([52529, 15, 6, 48])


In [15]:
from PatchTST_physics_integrated.models.stage2_long import (
    ReplicationPad1dPatchify,
    LongPatchEmbedding
)

patchify_long = ReplicationPad1dPatchify(patch_len=48, stride=24)
patch_embed_long = LongPatchEmbedding(patch_len=48, d_model=128)

patches_long = patchify_long(X_long)
tokens_long = patch_embed_long(patches_long)

print("tokens_long:", tokens_long.shape)


tokens_long: torch.Size([52529, 15, 6, 128])


In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/CSPatchTST/datasets/weather/weather.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)

feature_cols = [
    'p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
    'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)',
    'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)', 'wd (deg)', 'rain (mm)',
    'raining (s)', 'SWDR (W/m�)', 'PAR (�mol/m�/s)', 'max. PAR (�mol/m�/s)', 'Tlog (degC)'
]

X_all = df[feature_cols].astype("float32").values     # [N,20]
hour_all = df["date"].dt.hour.astype("int64").values # [N]


In [17]:
from torch.utils.data import DataLoader
from PatchTST_physics_integrated.data.weather_dataset import WeatherDataset

dataset = WeatherDataset(X_all, hour_all, context_window=168)

loader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

X_batch, hour_batch = next(iter(loader))
print(X_batch.shape, hour_batch.shape)


torch.Size([32, 168, 20]) torch.Size([32, 168])


In [18]:
from PatchTST_physics_integrated.models.stage2_short import (
    ReplicationPad1dPatchifyShort,
    ShortPatchEmbedding,
    ShortChannelTransformer
)

patchify_short = ReplicationPad1dPatchifyShort(patch_len=8, stride=4)
patch_embed_short = ShortPatchEmbedding(patch_len=8, d_model=128)
short_transformer = ShortChannelTransformer(d_model=128)

# get ONE batch only
X_batch, hour_batch = next(iter(loader))
X_long, X_short, X_aux = stage1(X_batch, hour_batch)

patches_short = patchify_short(X_short)
tokens_short  = patch_embed_short(patches_short)
H_short = short_transformer(tokens_short)

print("H_short:", H_short.shape)


H_short: torch.Size([32, 5, 41, 128])
